In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

digits = datasets.load_digits()

# choose some random images to display
indices = np.arange(len(digits.images))
display = np.random.choice(indices, size=5)

for i, image in enumerate(digits.images[display]):
    plt.subplot(1, 5, i+1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title("Label: %d" % digits.target[display[i]])
plt.show()
    
mnist_data = digits.images
mnist_data = mnist_data[:,np.newaxis,:,:]
labels = digits.target


<Figure size 640x480 with 5 Axes>

In [2]:
from sklearn.model_selection import train_test_split

train_to_test_ratio = 0.8
X_train, X_test, Y_train, Y_test = train_test_split(mnist_data, labels, train_size=train_to_test_ratio)


/home/andreas/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
from skimage.measure import block_reduce

class ConvolutionalNeuralNetwork:
    def __init__(self, X_data, Y_data):
        self.X_data = X_data
        self.Y_data = Y_data
        self.n_inputs, self.depth, self.input_width, self.input_height = X_data.shape
        
        self.eta = 0.1
        
        self.padding = 1
        self.receptive_field = 3
        self.stride = 1
        self.downsampling = 2
        
        self.n_filters_conv = 3
        self.n_neurons_flattened = int(self.n_filters_conv*self.input_width*self.input_height)
        self.n_neurons_connected = 10
        self.n_categories = 10
        
        self.output_width = int((self.input_width - self.receptive_field + 2 * self.padding) / self.stride + 1)
        self.output_height = int((self.input_height - self.receptive_field + 2 * self.padding) / self.stride + 1)
        self.downsampling_width = int(self.output_width / self.downsampling)
        self.downsampling_height = int(self.output_height / self.downsampling)
        
        self.create_biases_and_weights()

    def create_biases_and_weights(self):
        self.weights_conv = np.random.randn(self.n_filters_conv, self.depth, self.receptive_field, self.receptive_field)
        self.bias_conv = np.zeros(self.n_filters_conv)
        
        self.weights_connected = np.random.randn(self.n_neurons_flattened, self.n_neurons_connected)
        self.bias_connected = np.zeros(self.n_neurons_connected)
        
        self.weights_output = np.random.randn(self.n_neurons_connected, self.n_categories)
        self.bias_output = np.zeros(self.n_categories)
    
    def feed_forward(self):
        # Convolution layer
        self.z1 = np.zeros((self.n_inputs, self.n_filters_conv, self.output_width, self.output_height))
        for n in range(self.n_inputs):
            X_padded = np.pad(self.X_data[n,:,:,:], ((0,0),(self.padding,self.padding),(self.padding,self.padding)), 
                              'constant')
            for f in range(self.n_filters_conv):
                for w in range(self.output_width):
                    for h in range(self.output_height):
                        w1 = w*self.stride
                        w2 = w*self.stride + self.receptive_field
                        h1 = w*self.stride
                        h2 = w*self.stride + self.receptive_field
                        matrix_slice = X_padded[:,w1:w2,h1:h2]
                        self.z1[n, f, w, h] = np.sum(matrix_slice*self.weights_conv[f,:,:,:])
        self.a1 = np.maximum(self.z1, 0)
        
        # 2x2 downsampling layer
        self.z2 = np.zeros_like(self.a1)
        for n in range(self.n_inputs):
            for w in range(self.downsampling_width):
                for h in range(self.downsampling_height):
                    w1 = w*self.downsampling
                    w2 = w*self.downsampling + self.downsampling
                    h1 = h*self.downsampling
                    h2 = h*self.downsampling + self.downsampling
                    matrix_slice = self.a1[n,:,w1:w2,h1:h2]
                    matrix_slice[matrix_slice != matrix_slice.max()] = 0
                    self.z2[n,:,w1:w2,h1:h2] = matrix_slice
        self.a2 = np.maximum(self.z2, 0)
        
        # Fully connected layer
        self.downsampled_array = self.a2.reshape(-1, self.n_neurons_flattened)
        self.z3 = np.dot(self.downsampled_array, self.weights_connected) + self.bias_connected
        self.a3 = np.maximum(self.z3, 0)
        
        # Output layer
        exp_term = np.exp(self.a3)
        self.probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)
        self.probabilities[np.abs(self.probabilities) < 1E-16] = 0
        
    def feed_forward_out(self, X):
        # Convolution layer
        n_inputs = X.shape[0]
        z1 = np.zeros((n_inputs, self.n_filters_conv, self.output_width, self.output_height))
        for n in range(n_inputs):
            X_padded = np.pad(X[n,:,:,:], ((0,0),(self.padding,self.padding),(self.padding,self.padding)), 
                              'constant')
            for f in range(self.n_filters_conv):
                for w in range(self.output_width):
                    for h in range(self.output_height):
                        w1 = w*self.stride
                        w2 = w*self.stride + self.receptive_field
                        h1 = w*self.stride
                        h2 = w*self.stride + self.receptive_field
                        matrix_slice = X_padded[:,w1:w2,h1:h2]
                        self.z1[n, f, w, h] = np.sum(matrix_slice*self.weights_conv[f,:,:,:])
        a1 = np.maximum(z1, 0)
        
        # 2x2 downsampling layer
        z2 = np.zeros_like(a1)
        for n in range(n_inputs):
            for w in range(self.downsampling_width):
                for h in range(self.downsampling_height):
                    w1 = w*self.downsampling
                    w2 = w*self.downsampling + self.downsampling
                    h1 = h*self.downsampling
                    h2 = h*self.downsampling + self.downsampling
                    matrix_slice = a1[n,:,w1:w2,h1:h2]
                    matrix_slice[matrix_slice != matrix_slice.max()] = 0
                    z2[n,:,w1:w2,h1:h2] = matrix_slice
        a2 = np.maximum(self.z2, 0)
        
        # Fully connected layer
        downsampled_array = a2.reshape(-1, self.n_neurons_flattened)
        z3 = np.dot(downsampled_array, self.weights_connected) + self.bias_connected
        a3 = np.maximum(z3, 0)
        
        # Output layer
        exp_term = np.exp(a3)
        probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)
        probabilities[np.abs(probabilities) < 1E-16] = 0
        return probabilities
    
    def backpropagation(self):
        # Output layer
        error_output = self.probabilities
        error_output[range(self.n_inputs), self.Y_data] -= 1
        self.weights_output_gradient = np.dot(self.a3.T, error_output)
        self.bias_output_gradient = np.sum(error_output)
        
        # Fully connected layer
        error_connected = np.dot(error_output, self.weights_output.T)*(self.a3 > 0)
        self.weights_connected_gradient = np.dot(self.a2.T, error_connected).reshape(self.n_neurons_flattened, -1)
        self.bias_connected_gradient = np.sum(error_connected)
        
        # 2x2 downsampling layer
        error_downsampling = np.dot(error_connected, self.weights_connected.T)*(self.downsampled_array > 0)
        error_downsampling = error_downsampling.reshape(-1, self.n_filters_conv, self.output_width, self.output_height)
        
        # Convolutional layer
        error_conv = error_downsampling
        self.weights_conv_gradient = np.zeros_like(self.weights_conv)
        for n in range(self.n_inputs):
            X_padded = np.pad(self.X_data[n,:,:,:], ((0,0),(self.padding,self.padding),(self.padding,self.padding)), 
                              'constant')
            for f in range(self.n_filters_conv):
                for w in range(self.output_width):
                    for h in range(self.output_height):
                        w1 = w*self.stride
                        w2 = w*self.stride + self.receptive_field
                        h1 = w*self.stride
                        h2 = w*self.stride + self.receptive_field
                        matrix_slice = X_padded[:,w1:w2,h1:h2]
                        self.weights_conv_gradient[f,:,:,:] += X_padded[:,w1:w2,h1:h2]*error_conv[n, f, w, h]
        self.bias_conv_gradient = np.sum(error_conv)
        
        self.weights_output -= self.eta * self.weights_output_gradient
        self.bias_output -= self.eta * self.bias_output_gradient
        self.weights_connected -= self.eta * self.weights_connected_gradient
        self.bias_connected -= self.eta * self.bias_connected_gradient
        self.weights_conv -= self.eta * self.weights_conv_gradient
        self.bias_conv -= self.eta * self.bias_conv_gradient

In [4]:
import warnings
from tqdm import tqdm

warnings.filterwarnings("ignore")

cnn = ConvolutionalNeuralNetwork(X_train, Y_train)
for i in tqdm(range(10)):
    cnn.feed_forward()
    cnn.backpropagation()

pred = cnn.feed_forward_out(X_test)
print(pred[0].sum())


100%|██████████| 10/10 [00:31<00:00,  3.12s/it]


nan
